# 국민재난안전포털 웹 크롤러 만들기

In [7]:
from requests import Session
from requests.compat import urljoin
from bs4 import BeautifulSoup

In [5]:
session = Session()

base_url = 'http://www.safekorea.go.kr/'
main_url = 'idsiSFK/neo/ext/json/main/menuList.json'

main_resp = session.get(urljoin(base_url, main_url))
result = main_resp.json()

disaster_service_url = ''
disaster_menu_seq = 1
for cnt in range(0,len(result)):
    if cnt > 0 and result[cnt]['MENU_NM'] == '재난문자':
        disaster_service_url = 'idsiSFK' + result[cnt]['SERVICE_URL']
        disaster_menu_seq = result[cnt]['MENU_SEQ']


In [8]:
disaster_resp = session.get(urljoin(base_url, disaster_service_url))
dom = BeautifulSoup(disaster_resp.text, 'html.parser')

In [9]:
print(dom.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible">
   <meta content="국민재난안전포털" property="og:title"/>
   <meta content="" property="og:description"/>
   <meta content="https://www.safekorea.go.kr/idsiSFK/common/light/img/front/safekoreaDomain.jpg" property="og:image"/>
   <meta content="website" property="og:type"/>
   <meta content="#" property="og:url"/>
   <meta content="1539376542750650" property="fb:app_id"/>
   <title>
    국민재난안전포털
   </title>
   <link href="/idsiSFK/neo/ext/css/common.css" rel="stylesheet" type="text/css"/>
   <link href="/idsiSFK/favicon.ico" rel="shortcut icon"/>
   <!-- <link href="/idsiSFK/neo/ext/css/jquery-ui.css" rel="stylesheet" type="text/css"/> -->
   <!-- <link rel="stylesheet" type="text/css" href="/idsiSFK/neo/ext/css/front.min.css"/> -->
   <link href="/idsiSFK/neo/ext/css/layout.css" rel="stylesheet" type="text/css"/>
   <!-- <link href="/idsiSFK/neo/ext/css/dp_sub.css" rel="st